![Nuclio logo](https://nuclio.school/wp-content/uploads/2018/12/nucleoDS-newBlack.png)

# Crimes in Boston

Times, Locations and descriptions of crimes

## Table of Contents

* [A. Introduction](#introduction)
* [B. Importing Libraries](#libraries)
* [C. Importing data](#data)
* [MLC 2: Data understanding](#data_understanding)
    * [MLC 2.1.: Univariate data analysis](#univariate_data_analysis)
        * [2.1.1 - 2.1.2. Dataset size and direct visualization of the data](#dataset_size_and_visualization)
        * [2.1.3. Types of variables available](#variable_types)
        * [2.1.4. Descriptive statistics](#descriptive_statistics)
        * [2.1.5. Null values](#nulls)
        * [2.1.6. Identification of outliers](#outliers_identification)
        * [2.1.7. Identification of errors in the data](#errors) 
        * [2.1.8. Visualization of distributions](#data_distributions) 
* [D. Data Insights](#insights)
* [MLC 3: Data preparation](#data_preparation)
    * [MLC 3.1. Data cleaning](#data_cleaning)
        * [3.1.1. Imputation of null values](#nulls_imputation)            
        * [3.1.2. Handling outliers](#handling_outliers)
        * [3.1.3. Dealing with variable types](#dealing_variable_types)
        * [3.1.4. Elimination of features with low variance or highly correlated with others](#low_variance)
    * [3.2 - MLC 3.4: Data Transformation, Feature Engineering and Data Selection](#data_transformation_engineering_selection)
* [MLC 4: Modelling and Evaluation](#modelling_evaluation)
    * [4.1. K-Means](#kmeans)
    * [4.2. DBSCAN](#dbscan)
    * [4.3. Birch](#birch)
* [E. Next steps - What to try next](#next_steps)

Note: Take into account that the ML checklist has been also adapted to the needs of the specific dataset and to support a better storytelling throughout this notebook. Some sections may not follow the expected order or some operations may have been done in an earlier stage after confirming these were possible.

## A. Introduction <a class="anchor" id="introduction"></a>

We are working with the BPD (Boston Police Department) with the task to helping to derive some insights from the crime data they provided. Our work is to include all stages of the ML Checklist (data understanding, data preparation, graphs and clusters) and create a clear and defined story over it.

## B. Importing Libraries <a class="anchor" id="libraries"></a>

In [ ]:
# system and url requests
import os.path
import requests

# removing system warnings
import warnings
warnings.filterwarnings('ignore')

# data manipulation 
import pandas as pd
import numpy as np

#preprocessing
from sklearn.preprocessing import StandardScaler

# modelling
from sklearn import model_selection
from sklearn.neighbors import KNeighborsClassifier
from sklearn.cluster import KMeans, DBSCAN, Birch

# plotting
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from wordcloud import WordCloud

# plotting options
%matplotlib inline
plt.style.use('ggplot')
plt.rcParams["figure.figsize"] = (10, 7)

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.options.display.float_format = '{:,.2f}'.format

# constants
SEED = 42

## C. Importing data<a class="anchor" id="data"></a>

In [ ]:
# download Google Drive files helper functions
# useful as it doesn't use any Google third party libs, as we only want to download the data
# Adapted from https://stackoverflow.com/questions/25010369/wget-curl-large-file-from-google-drive/39225039#39225039

def get_file_id(gdrive_url):
    """
    Extracts the file ID from a Google Drive URL
    """
    return gdrive_url.split('/')[-2]

def get_confirm_token(response):
    """
    Gets the randomly generated confirm token by Google Drive
    """
    for key, value in response.cookies.items():
        if key.startswith('download_warning'):
            return value
    return None

def save_response_content(response, destination):
    """
    Saves file contents from the response in the given destination
    """
    CHUNK_SIZE = 32768
    with open(destination, "wb") as f:
        for chunk in response.iter_content(CHUNK_SIZE):
            if chunk: # filter out keep-alive new chunks
                f.write(chunk)

def download_file_from_google_drive(gdrive_url, destination):
    """
    Downloads and stores the file from the given url in the destination folder
    """
    URL = "https://docs.google.com/uc?export=download"
    session = requests.Session()
    id = get_file_id(gdrive_url)
    response = session.get(URL, params = { 'id' : id }, stream = True)
    token = get_confirm_token(response)
    if token:
        params = { 'id' : id, 'confirm' : token }
        response = session.get(URL, params = params, stream = True)
    save_response_content(response, destination)

def download_files(files_list, destination_directory):
    """
    Downloads a list of files from a 'file_list' dictionary into a 'destination_directory'.

    'file_list' is a list of dictionaries with the following mandatory keys:
        - 'name': Name of the file
        - 'url': URL from which the file should be downloaded

    Returns a list of directories where the downloaded files can be found
    """
    dir_list = []
    for file in files_list:
        # check if destination_folder exists and creates it otherwise
        if not os.path.exists(destination_directory):
            print("Creating", destination_directory, "directory", end = '\n\n')
            os.makedirs(destination_directory)

        file_path = './' + destination_directory + '/' + file['name']
        if not os.path.isfile(file_path):
            print(file['name'])
            print("Downloading from:", file['url'], end = '\n\n')
            download_file_from_google_drive(file['url'], file_path)
        else:
            print(file['name'], "already inside of", destination_directory)
        
        dir_list.append(file_path)
    return dir_list

In [ ]:
# downloading the files from Google Drive URLs
directory = 'data/original'

files_to_download = [
  {
    'name': 'crime.csv',
    'url': 'https://drive.google.com/file/d/11y05JjYleaA0FRB4448GH5EKka-1C26Z/view?usp=sharing'
  },
  {
    'name': 'offense_codes.csv',
    'url': 'https://drive.google.com/file/d/122eCg0ieLEAEY95mKuwRjJb0xztBalW_/view?usp=sharing'
  }
]

file_dirs = download_files(files_to_download, directory)

In [ ]:
# importing data into Pandas DataFrames
crimes = pd.read_csv(file_dirs[0], encoding = 'windows-1252')
oc = pd.read_csv(file_dirs[1], encoding = 'windows-1252')

## MLC 2: Data Understanding<a class="anchor" id="data_understanding"></a>

### MLC 2.1: Univariate data analysis<a class="anchor" id="univariate_data_analysis"></a>

First of all, let's review the 2 datasets we generated `crimes` and `oc` to see with what type of data we are dealing with.

#### 2.1.1 - 2.1.2. Dataset size and direct visualization of the data<a class="anchor" id="dataset_size_and_visualization"></a>

Let's review both available datasets in a little bit more detail.

#### `crimes`

In [ ]:
crimes.head()

In [ ]:
crimes.columns

Each row in the `crimes` dataset contains a crime report, including the type, the district and street where it happened, day and time, geospatial location...

In [ ]:
print("There are", crimes.shape[0], "records, and", crimes.shape[1], "columns to review")

In [ ]:
print("Crime records go from", crimes.OCCURRED_ON_DATE.min(), "until", crimes.OCCURRED_ON_DATE.max())

In [ ]:
crimes.info(verbose = True)

#### `oc` (Offense Codes)

In [ ]:
oc.head()

In [ ]:
oc.columns

Each row in the `oc` DataFrame corresponds to an Offense Code Number and its Name (or description). This may be useful in order to understand what does the `OFFENSE_CODE` mean in the `crimes` dataset.

In [ ]:
print("There are", oc.shape[0], "records, and", oc.shape[1], "columns to review")

In [ ]:
oc.info(verbose = True)

#### 2.1.3. Types of variables available<a class="anchor" id="variable_types"></a> 

##### Merging `oc` with `crimes`

Before starting with any analysis on the crimes dataset, we will see if it's possible to merge the `oc` data with the `crimes` dataset, as it seems that, when inspecting the data in `crimes` there may be some columns with repeated information that we could remove (mainly `OFFENSE_CODE`, `OFFENSE_CODE_GROUP`, `OFFENSE_DESCRIPTION`).

In [ ]:
crimes.OFFENSE_CODE.nunique()

In [ ]:
oc.CODE.nunique()

It seems that we have more Offense Codes in `oc` than in crimes. However, this may mean that we have unused values in `crimes`, which is fine. Nonetheless, what is kind of worrying is that there are 425 unique codes in `oc`, but we have 576 records, so it means that we have some duplicates. Let's check them out.

In [ ]:
oc.sort_values('CODE').head(20)

By inspecting the data we see, for example that for `CODE = 111` we have 2 entries which mean the same and it only changes a comma. For `CODE = 112` we have basically the same info. So we'll drop duplicates and we'll keep the first entry we encounter. Additionally, we have seen that there may be values with commas (`CODE = 111`), this is something to take into account for as soon as we merge both DataFrames and checking the result.

In [ ]:
# removing commas, so we are getting a standardized NAME column
oc.NAME = oc.NAME.str.replace(',', '')

In [ ]:
# dealing with duplicate codes
oc.drop_duplicates(subset = 'CODE', keep = 'first', inplace = True)

In [ ]:
oc.shape

Now we can merge `oc` code names with `crimes`

In [ ]:
oc.columns = ['OFFENSE_CODE', 'OFFENSE_NAME']
crimes = crimes.merge(oc, on='OFFENSE_CODE', how='inner')

By simple inspection, we can see that maybe `OFFENSE_DESCRIPTION` and `OFFENSE_NAME` are the same, so let's double check.

In [ ]:
crimes.OFFENSE_DESCRIPTION.nunique()

In [ ]:
crimes.OFFENSE_NAME.nunique()

There are less unique values in the newly created column than in the old one. Checking if this is due to null values:

In [ ]:
crimes.isna().sum()

That's not the case, there are no null values in these columns... so let's inspect the values that are different between these 2 columns.

In [ ]:
# removing also the commas from OFFENSE_DESCRIPTION so we can compare the values properly
crimes.OFFENSE_DESCRIPTION = crimes.OFFENSE_DESCRIPTION.str.replace(',', '')

In [ ]:
crimes.loc[crimes['OFFENSE_NAME'] != crimes['OFFENSE_DESCRIPTION'], ['OFFENSE_DESCRIPTION', 'OFFENSE_NAME']].drop_duplicates()

Seems that in the fields where there are differences, these are very similar between them, so... why is that? Let's do some more analysis to see if there are any repeated values in `OFFENSE_DESCRIPTION`.

In [ ]:
print(crimes.loc[crimes['OFFENSE_DESCRIPTION'] == 'LARCENY ALL OTHERS', ['OFFENSE_DESCRIPTION']].count())
print(crimes.loc[crimes['OFFENSE_DESCRIPTION'] == 'LARCENY OTHER $200 & OVER', ['OFFENSE_DESCRIPTION']].count())

In [ ]:
print(crimes.loc[crimes['OFFENSE_DESCRIPTION'] == 'LARCENY NON-ACCESSORY FROM VEH. $200 & OVER', ['OFFENSE_DESCRIPTION']].count())
print(crimes.loc[crimes['OFFENSE_DESCRIPTION'] == 'LARCENY THEFT FROM MV - NON-ACCESSORY', ['OFFENSE_DESCRIPTION']].count())

By looking at a couple of the fields, we see that `OFFENSE_DESCRIPTION` has not only some fields with the same values as in `OFFENSE_NAME` but also some other fields which are causing the differences. So, in order to keep our data the most standardized possible, we will remove the `OFFENSE_DESCRIPTION` column and keep the one we generated from the `oc` DataFrame.

In [ ]:
crimes.drop(columns = 'OFFENSE_DESCRIPTION', axis = 1, inplace = True)

From now on, we won't be using `oc` as a DataFrame and we'll stick with `crimes`.

##### Checking variables data types

In [ ]:
crimes.info(verbose = True)

In [ ]:
crimes.dtypes

In [ ]:
# standardizing column names to uppercase
crimes.columns = [col.upper() for col in crimes.columns]
crimes.dtypes

We have the following variables in `crimes`:

- Categorical
  - `INCIDENT_NUMBER` - It represents the ID or number in the incident register        
  - `OFFENSE_CODE_GROUP` - Groups the offenses from `OFFENSE_NAME` in a category  
  - `OFFENSE_NAME` - Description for an Offense, given by its direct relationship to the `OFFENSE_CODE`
  - `DISTRICT` - Boston City district where the incident occurred               
  - `REPORTING_AREA` - Boston City Area Code where the offense occurred         
  - `SHOOTING` - Boolean variable that identifies if there was a shooting in the incident or not.               
  - `OCCURRED_ON_DATE` - Date and time on when the offense occurred       
  - `DAY_OF_WEEK` - Day of the week (Monday - Sunday) of when the incident occurred            
  - `UCR_PART` - Uniform Crime Report [+info](https://en.wikipedia.org/wiki/Uniform_Crime_Reports)          
  - `STREET` - Boston Street Name where the incident occurred                 
  - `LOCATION` - Tuple identifying Latitude and Longitude for the location of the crime               
- Numerical
  - `OFFENSE_CODE` - ID/Code that identifies the offense. It's highly related to `OFFENSE_NAME`          
  - `YEAR` - Year when the crime occurred                  
  - `MONTH` - Month when the crime occurred                 
  - `HOUR` - Hour (without minutes) when the crime occurred                  
  - `LAT` - Latitude component of the location of the crime                  
  - `LONG` - Longitude component of the location of the crime                 


#### 2.1.4. Descriptive statistics<a class="anchor" id="descriptive_statistics"></a> 

##### Categorical values

In [ ]:
crimes.describe(include = 'object').T

Something interesting things are that:
- `INCIDENT_NUMBER` even though it should be an ID, it has repeated values. We will need to review the dataset to see if there are duplicated entries to remove.
- `SHOOTING` has only 1 unique value, when it seems to be a boolean variable. However, this will become more clear when we check for nulls

##### Numerical values

In [ ]:
crimes.describe(include = 'number').T

#### 2.1.5. Null values<a class="anchor" id="nulls"></a> 

In [ ]:
# categorical values
crimes.select_dtypes(include=['object']).isnull().sum()

In [ ]:
# numerical
crimes.select_dtypes(include=['number']).isnull().sum()

The following columns have null values that we'll need to sort out:
- Categorical
  - `DISTRICT`
  - `SHOOTING`
  - `UCR_PART`
  - `STREET`
- Numerical
  - `LAT`
  - `LONG`

#### 2.1.6. Identification of outliers<a class="anchor" id="outliers_identification"></a> 

In [ ]:
# plotting helping functions

def autopct_generator(limit):
  """
  Hides percentages in bar charts given a specific 'limit'
  """
  def inner_autopct(pct):
      return ('%.0f%%' % pct) if pct > limit else ''
  return inner_autopct

def get_new_labels(values, labels, limit):
  """
  Returns a list of labels only if they exceed a specific 'limit'
  given 'values' and 'labels' lists for a pie chart
  """
  return [label if value > limit / 100 else '' for value, label in zip(values, labels)]

def add_subplot(df, column, plot_type, axes, styles):
  """
  Adds a subplot using the data from a 'df' 'column' in the position given by its 'axes'.
  Types available: Pie, Histogram, Bar, Box.
  Optionally a 'styles' dictionary can be added in order to change the plot styles.
  """
  axes.set_title(column, fontsize = 12)
  values_series = df[column].value_counts(dropna = False, normalize = True)
  if plot_type == 'Pie':
    #axes.pie(values_series.values, labels = values_series.index, autopct=autopct_generator(10))
    axes.pie(values_series.values, labels = get_new_labels(values_series.values, values_series.index, styles['limit']), autopct = autopct_generator(styles['limit']))
  elif plot_type == 'Hist':
    sns.histplot(df, x = column, ax = axes, discrete = styles['discrete'])
  elif plot_type == 'Bar':
    sns.barplot(x = values_series.index, y=values_series.values, ax = axes)
  elif plot_type == 'Box':
    sns.boxplot(data = df, y = column, ax = axes)
  else:
    print("Please specify a valid plot_type: ['Pie', 'Hist', 'Bar', 'Box]")

def clean_unused_subplots(fig, axes, row, col, plot_size):
  """
  From a given a subplot 'fig' of size 'plot_size', it removes non-used 'axes' 
  starting at the given 'row', 'col' coordinates
  """
  while row < plot_size[0]:
    while col < plot_size[1]:
      fig.delaxes(axes[row, col])
      col +=1
    row +=1

def generate_subplots(df, subplot_type, n_cols = 3, styles = {}):
  """
  Generates an homogeneous subplot of all columns in a dataframe 'df'.
  Setting the max number of columns n_cols, the subplot is automatically arranged for 
  a better display.
  Types available: Pie, Histogram, Bar, Box.
  """
  styles = {**{ 'size': (40, 50), 'n_bins': 10, 'discrete': False, 'limit': 10 }, **styles}
  # auto-scaling the subplot based on the number of columns
  n_rows = int(np.ceil([len(df.columns) / n_cols])[0])
  # setting up the subplot
  fig, axes = plt.subplots(n_rows, n_cols, figsize= styles['size'], constrained_layout = True)
  fig.suptitle(styles['title'], fontsize = 22)
  # plotting variables from df
  row = -1
  col = -1
  for idx, column in enumerate(df.columns):
    # managing layout
    col = (idx % n_cols)
    if col == 0:
      row += 1
    # adding subplots
    if n_rows == 1:
      add_subplot(df, column, subplot_type, axes[col], styles)
    else:  
      add_subplot(df, column, subplot_type, axes[row, col], styles)
  clean_unused_subplots(fig, axes, row, col + 1, (n_rows, n_cols))

In [ ]:
styles = {
  'title' : 'Numerical variables boxplots for outliers detection',
  'size' : (10, 6)
}

generate_subplots(crimes.select_dtypes(include = 'number'), 'Box', 3, styles) # removing TARGET from analysis

We can see that only latitude and longitude have outliers. However, by inspecting the data again, we see that the column `LOCATION` has no nulls and also it has a pair of coordinates that correspond to the Latitude and Longitude too (numbers are very similar)

In [ ]:
crimes[['LAT', 'LONG', 'LOCATION']].head(10)

`LOCATION[0]` is very similar to `LAT` and `LOCATION[1]` is very similar to `LONG`.

In [ ]:
# comparing nulls between the 3 columns
crimes[['LAT', 'LONG', 'LOCATION']].isna().sum()

In [ ]:
crimes[['LAT', 'LONG', 'LOCATION']].loc[(crimes['LAT'].isna()) | (crimes['LONG'].isna())].head(10)

We can observe, that the places where `LAT` and `LONG` are nulls, is also true that this is because we have no location available. 

Additionally, let's confirm if there's any significant difference in these values, so we can decide if wether we remove the column `LOCATION` or if there's any value in keeping it. 

In [ ]:
# converting locations into tuples
crimes.LOCATION = crimes.LOCATION.apply(lambda x: tuple(x[1: -1].split(','))) # using [1: -1] to remove the '(' and ')'

In [ ]:
# extracting LAT and LONG from LOCATION
location_LAT = crimes.LOCATION.apply(lambda x: float(x[0]))
location_LONG = crimes.LOCATION.apply(lambda x: float(x[1]))

In [ ]:
# extracting LAT and LONG from LOCATION
location_LAT = crimes.LOCATION.apply(lambda x: float(x[0]))
location_LONG = crimes.LOCATION.apply(lambda x: float(x[1]))

# comparing extracted LAT and LONG with LOCATION
locations = crimes[['LAT', 'LONG']]

locations.loc[:, 'COMP_LAT'] = location_LAT
locations.loc[:, 'COMP_LONG'] = location_LONG

# calculating the error
locations.loc[:, 'ERROR_LAT'] = locations.LAT - locations.COMP_LAT
locations.loc[:, 'ERROR_LONG'] = locations.LONG - locations.COMP_LONG

print(f"The aggregated error of COMP_LAT with the real LAT is {locations.ERROR_LAT.sum()}")
print(f"The aggregated error of COMP_LONG with the real LONG is {locations.ERROR_LAT.sum()}")

In [ ]:
del(location_LAT, location_LONG, locations)

So, we can remove the `LOCATION` column in `crimes`, as we already have its split features `LAT` and `LONG` which correspond to the same ones we have there.

In [ ]:
crimes.drop(columns = 'LOCATION', axis = 1, inplace = True)

#### 2.1.7. Identification of errors in the data<a class="anchor" id="errors"></a> 

##### `INCIDENT_NUMBER` has repeated values

`INCIDENT_NUMBER`, by definition, should be an identifier of each crime offense, so having repeated values is not correct. Let's check this phenomena to understand what's going on.

In [ ]:
print(f"There are {len(crimes) - crimes.INCIDENT_NUMBER.nunique()} repeated INCIDENT_NUMBER ({round((len(crimes) - crimes.INCIDENT_NUMBER.nunique()) / len(crimes) * 10, 2)} % of total rows)")

In [ ]:
# checking duplicates
duplicates = crimes[crimes.duplicated(keep = False)]
len(duplicates)

In [ ]:
# removing duplicates
crimes.drop_duplicates(keep = "first", inplace = True)

However, it seems that there are only 64 duplicates, understanding these as rows that are 100% equal in all their columns. Seems that we'll need to dig deeper.

In [ ]:
crimes.INCIDENT_NUMBER.value_counts(dropna = False).head(10)

There are several Incident Numbers that are repeated. So, let's inspect them to see with what kind of issue we are dealing with.

In [ ]:
crimes.loc[crimes["INCIDENT_NUMBER"] == 'I162030584'].sort_values(by = "INCIDENT_NUMBER")

In [ ]:
crimes.loc[crimes["INCIDENT_NUMBER"] == 'I152080623'].sort_values(by = "INCIDENT_NUMBER")

As we can see, these are crimes occurred in the same place of the incident. All column values are equal except the `OFFENSE_CODE`, the `OFFENSE_CODE_GROUP` and the `OFFENSE_NAME`. So, even though in the description, `INCIDENT_NUMBER` is understood as each incident that occurred, it seems that each incident can have several crime records or offenses related to it. 

One way to treat this would be by merging all these offenses together in lists and generate only 1 incident with all offenses listed inside it (the first one will come with index 0, the second with index 1...). However, since we are looking at an Unsupervised Learning algorithm and we want to generate clusters out from the data, it's better to keep each offense separated, even if it occurs in the same place. At least for now.

So, this column is going to be useless, not only as an index, but also as information for a future model, so we'll drop it... but later, just in case it could be useful to correct some data.

##### Treating `OCCURRED_ON_DATE` as a Datetime variable and not as categorical

`OCCURRED_ON_DATE` is considered as an object/categorical value, we will convert it to a `datetime` variable so we can consider it as such throughout the analysis

In [ ]:
crimes.OCCURRED_ON_DATE = pd.to_datetime(crimes.OCCURRED_ON_DATE)

#### 2.1.8. Visualization of distributions<a class="anchor" id="data_distributions"></a> 

#####  Categorical

In [ ]:
crimes.describe(include = 'object').columns

In [ ]:
styles = {
  'title' : 'Pie charts for categorical values distributions',
  'size' : (18, 10),
  'limit': 5 # sets the lower percentage limit from which the pie charts won't show the label nor the percentage in the subplots
}

# removing 'INCIDENT_NUMBER' as it has too many values to display
generate_subplots(crimes[['OFFENSE_CODE_GROUP', 'DISTRICT', 'REPORTING_AREA', 'SHOOTING', 'DAY_OF_WEEK', 'UCR_PART', 'STREET', 'OFFENSE_NAME']], 'Pie', 4, styles)

Some comments when looking at the data distributions:

- `OFFENSE_CODE_GROUP`: There are many categories with a top 3 of `Motor Vehicle Accident Response`, `Larceny` and `Medical Assistance`. The top 3 categories correspond to the 27% of all values, but then, there are quite some other categories to review later.
- `DISTRICT`: It's quite similar to `OFFENSE_CODE_GROUP` although the distribution is better as not many values concentrate above 5% of the total.
- `REPORTING AREA`: There's a huge dispersion on the values, as soon as we start analysing the data and clustering it, this may be interesting. However, it's weird that we see that the Area with highest amount of offenses is reported as `''` (empty).
- `SHOOTING`: Almost 100% of the data is `NaN`. As soon as we review and correct the null values, we'll check what may be happening there.
- `DAY_OF_WEEK`: The distribution is pretty similar among the different days of the week.
- `UCR_PART`: It has 3 distinguishable parts.
- `STREET`: None of the streets reaches the minimum of 5% to be shown in the pie chart (due to the plotting function configuration), and there are tons of them.
- `OFFENSE_NAME`: The top 3 values represent around 17% of all the values, but there are many names to be considered when we explore the data further.

#####  Numerical

In [ ]:
crimes.describe(include = 'number').columns

In [ ]:
styles = {
  'title' : 'Histograms for categorical values distributions',
  'size' : (10, 6)
}

generate_subplots(crimes.select_dtypes(include = 'number'), 'Box', 3, styles)

Some comments when looking at these distributions:
- `OFFENSE_CODE` has its median around 2900 and its variance between quantile 25 and 75 is pretty wide.
- `YEAR`, as we know, we only have data from 2015 till 2018, and the median is around 2017.
- `MONTH`, as it's obvious, it has only values from 1 (January) up until 12 (December), with its median on July (7).
- `HOUR`, as expected, hours range from 0 to 24. And its median is around 14h.
- `LAT` and `LONG` have a very thin IQR and pretty differentiated outliers.

Let's look at the time distributions a little bit in more detail:

In [ ]:
def plot_hist_with_mean(df, x, title):
    """
    Given a DataFrame 'df' and a variable or col 'x', the funcion 
    generates and plots a Histogram with its mean.

    Additionally, a 'title' to the plot can be given. 
    """
    # setting histogram
    fig = px.histogram(
        df, 
        x = x, 
        title = title
    )

    # calculating the mean of the graph and plotting
    no_nulls_col = ''
    for col in df.columns:
        if not df[col].isna().any():
            no_nulls_col = col
            break
    mean = df.groupby(x).count()[no_nulls_col].mean()
    fig.add_hline(
        y = mean,
        line_color = "red",
        annotation_text = "Avg:" + str('{:,}'.format(round(mean, 2))), 
        annotation_position = "top right",
        annotation_font_size = 14,
        annotation_font_color = "red",
    )

    # update layout and display the plot
    fig.update_layout(
        xaxis = dict(tickmode = 'linear'), 
        bargap = 0.01
    )
    fig.show()

`YEAR`

In [ ]:
plot_hist_with_mean(crimes, 'YEAR', 'Crime Incidents per Year')

From the data we have, the vast majority of the crimes are concentrated between 2016 and 2017.

`MONTH`

In [ ]:
plot_hist_with_mean(crimes, 'MONTH', 'Crime Incidents per Month')

Crimes are usually below the average number, except during the summer months, were criminality is higher (May, June, July, August and September).

`HOUR`

In [ ]:
plot_hist_with_mean(crimes, 'HOUR', 'Crime Incidents per Hour of the Day')

The majority of crimes occur during the day, from 9 till 21 hours more or less. Seems that law offenders like to sleep too. There's also a peak at 12 AM, which may be due to either outliers or missing data, which has been imputed in the first available hour. However, we have no additional data to see if that's the case or not, so we'll leave it like that.

## D.  Data Insights<a class="anchor" id="insights"></a>

In the following section we are going to extract some data insights and answer preliminary questions in order to help the Boston Police Department to identify crime faster in Boston City. Some data transformations may be done along the way which will ease our preparation for the future model.

In [ ]:
# transforming OCCURRED_DATE to have several new fields for date analysis
crimes['DAY'] = crimes.OCCURRED_ON_DATE.dt.day
crimes['MINUTE'] = crimes.OCCURRED_ON_DATE.dt.minute
crimes['QUARTER'] = crimes.OCCURRED_ON_DATE.dt.quarter
crimes['WEEK'] = crimes.OCCURRED_ON_DATE.dt.isocalendar().week

crimes.head(1)

### What are the distributions of crimes throughout time?

Let's analyse how time affects the amount of crimes in Boston. When should the Boston Police Department have more agents on the streets according to the data available?

By looking at the years where we have complete data (2016, 2017) let's try to see the crime distribution from several angles.

In [ ]:
fig = px.histogram(crimes[crimes.YEAR.isin([2016, 2017])], x = 'QUARTER', color = 'YEAR', title = 'Crime histogram per quarter (2016 - 2017)')
fig.update_layout(bargap = 0.01)
fig.show()

In [ ]:
fig = px.histogram(crimes[crimes.YEAR.isin([2016, 2017])], x = 'MONTH', color = 'YEAR', title = 'Crime histogram per Month (2016 - 2017)')
fig.update_layout(bargap = 0.01)
fig.show()

In [ ]:
fig = px.histogram(crimes[crimes.YEAR.isin([2016, 2017])], x = 'DAY', color = 'MONTH', title = 'Crime histogram per day of the month (2016 - 2017)')
fig.update_layout(bargap = 0.01)
fig.show()

In [ ]:
fig = px.histogram(crimes[crimes.YEAR.isin([2016, 2017])], x = 'DAY_OF_WEEK', color = 'YEAR', title = 'Crime histogram per day of the week (2016 - 2017)')
fig.update_layout(bargap = 0.01)
fig.show()

From the following data we can infer that:
- Throughout the years, the crime distribution has been kept similar, so there are no years with higher criminality rates.
- Quarter 3 (in 1st place) and Quarter 2 (in 2nd place) represent the times of the year where there are more crimes.
- The first day of the month is the one that has a higher number of crimes occurring. If this is due to the same reason on why some crimes have been imputed to the first hour of the day or not, this is unknown.
- Crimes are kind of evenly distributed from Monday till Sunday. However, there's a slight peak on Fridays (4).

### What are the most dangerous streets and districts?

Let's analyse the streets and districts where the crime indices are higher.

In [ ]:
fig = px.histogram(crimes, x = 'DISTRICT', title = 'Districts by Number of Crimes').update_xaxes(categoryorder = 'total descending')
fig.show()

District `B2` is the one that has had more crimes reported, and `A15` seems to be the "safest". Since Boston has 12 districts, it seems that there's no district that has had no crimes during these years.

Let's check how do streets look like:

In [ ]:
print("Unique STREET values: ", crimes.STREET.nunique()) 

Since there are so many STREET values, we will limit the visualization to the top 100 most dangerous streets, in terms of crimes occurred on them.

In [ ]:
top_100_streets = crimes.STREET.value_counts()[:100].index.to_list()
px.histogram(crimes[crimes.STREET.isin(top_100_streets)], x = 'STREET', title = 'Streets by Number of Crimes').update_xaxes(categoryorder = 'total descending')

In [ ]:
# another way of representing this informaton is by using a word cloud
text = ', '.join([street.replace(' ', '_') for street in crimes.loc[crimes.STREET.isin(top_100_streets), 'STREET']])
wordcloud = WordCloud(max_font_size = 50, max_words = 100, background_color = 'white').generate(text)

plt.figure()
plt.imshow(wordcloud, interpolation = 'bilinear')
plt.axis('off')
plt.show()

We can see that `WASHINGTON ST`, `BLUE HILL AVE`, `DORCHESTER AVE` and `BOYLSTON ST` are among the most dangerous streets in Boston. However, from the previous analysis, we should take into account that the `NaN` are the second highest occurrence in street names where crimes occurred. If we could get the BPD to provide this missing data, that could help on better increasing the weights of other streets that may look more secure than what they really are.

### How many shootings have there been in Boston?

Even though Boston hasn't had many shootings, let's focus on them.

In [ ]:
shootings = crimes[crimes.SHOOTING == 'Y']
print(f"From the data we have, there have been {shootings.shape[0]} shootings from {shootings.YEAR.min()} till {shootings.YEAR.max()} ({round(shootings.shape[0] / crimes.shape[0], 4)}% of total crimes)")

Let's check when they are occurring the most:

In [ ]:
fig = px.histogram(shootings, x = 'MONTH', color = 'YEAR', title = "Shootings per month")
fig.update_layout(bargap = 0.01)
fig.show()

In [ ]:
fig = px.histogram(shootings, x = 'DAY_OF_WEEK', color = 'MONTH', title = "Shootings per day of week").update_xaxes(categoryorder = 'total descending')
fig.update_layout(bargap = 0.01)
fig.show()

It seems that shootings mainly occur mainly on Saturdays. 

What if we check how are shootings distributed throughout districts?

In [ ]:
fig = px.histogram(shootings, x = 'DISTRICT', title = 'Shootings by District').update_xaxes(categoryorder = 'total descending')
fig.show()

In terms of shootings, district `B2` is still the one with the highest shooting rates, but then, `B3` appears as the second district with most of the shootings, when this district was the 5th in terms of total crimes. So it has a higher percentage of shootings over the total amount of crimes.

On the other hand, district `A1`, which has quite a low amount of shootings has a lot of crimes (it's number 4 in total amount), which means that the crimes there don't involve a huge amount of violence and gunfire, but there's still quite a lot of crime happening.

### What are the most common crime types?

Let's check which are the most common types of crimes in Boston and how are shootings related to them.

In [ ]:
fig = px.histogram(crimes, x = 'OFFENSE_CODE_GROUP', title = "Most common Offenses in Boston").update_xaxes(categoryorder = 'total descending')
fig.show()

In [ ]:
fig = px.histogram(shootings, x = 'OFFENSE_CODE_GROUP', title = "Most common Offenses (involving shootings) in Boston").update_xaxes(categoryorder = 'total descending')
fig.show()

The most common crime is `Motor Vehicle Accident Response`, but this one usually doesn't involve a shooting. However, when there's an `Aggravated Assault` it's when it's more likely to have a shooting situation, but this only occurs in 521 over 7792 cases (6.7 %).

## MLC 3: Data preparation<a class="anchor" id="data_preparation"></a> 

### MLC 3.1. Data cleaning<a class="anchor" id="data_cleaning"></a> 

#### 3.1.1. Imputation of null values<a class="anchor" id="nulls_imputation"></a>

In [ ]:
crimes.isna().sum()

`DISTRICT`

In [ ]:
crimes.DISTRICT.value_counts(dropna = False)

There are 13 different values: 12 Districts and NaN. Investigating a little bit, here you can see that there are 12 [Boston Districts](https://bpdnews.com/districts), so the nulls can't be imputed to any district with the info we have.

Let's try the approach to use other variables, such as the `LAT` and `LONG`, to see if we can place the missing records into specific districts.

In [ ]:
crimes.DISTRICT.fillna('UNDEFINED', inplace = True) # filling NaNs with 'UNDEFINED', for now, for better analysis and searchability inside the DataFrame
crimes.pivot_table(index ='DISTRICT', values = ['LAT', 'LONG'], aggfunc=[len, np.mean])

Using a pivot table and checking the mean, we see that the `UNDEFINED` districts may also be incorrect in terms of `LAT` and `LONG`. All other districts look alike and coordinates are pretty similar and very centered to the right location of Boston ((42.35843, -71.05977) according to [this source](https://latitude.to/map/us/united-states/cities/boston)), but the `UNDEFINED` districts are pretty far away from the others, so these could be either errors in data or due to null values. Let's inspect them a little bit more.

In [ ]:
# checking what the values of 'LAT' and 'LONG' the unknown districts have
crimes.loc[crimes.DISTRICT == 'UNDEFINED', ['LAT', 'LONG']].head(10)

Inspecting the data, we see that there are many entries where `LAT` and `LONG` is `-1`, which correspond to the outliers we detected (and also seem incorrect taking into account the mean of the coordinates). Nonetheless, there are also some entries that look correct and maybe could be pinpointed to a specific district, but we need a more graphical approach to be able to see it. 

Let's check some of these entries to see if we can use other columns to pinpoint them to a district.

In [ ]:
crimes.iloc[[120, 203, 214, 248]]

By inspecting the data, the only field we would be able to use in order to locate the `DISTRICT` (or `LAT` and `LONG`) is the `STREET`, but since we don't have the full address, that may be complicated because some streets could be long enough to go through several districts. Before taking any decision, let's take a look at what we are dealing with, because depending on the number of streets, we could quickly search into Google and identify where they are.

In [ ]:
crimes.shape

In [ ]:
# checking all 'UNDEFINED' DISTRICTS where we don't have  additional information to locate the reporting area
unknown_area = ((crimes.LAT == -1) | (crimes.LAT.isna())) & ((crimes.LONG == -1) | (crimes.LONG.isna())) & (crimes.DISTRICT == 'UNDEFINED') & (crimes.STREET.isna())
crimes[unknown_area].head(10)

In [ ]:
print(f"There are {crimes[unknown_area].shape[0]} entries to remove ({round(crimes[unknown_area].shape[0] / crimes.shape[0], 4)} %)")

In [ ]:
# dropping these rows as we don't have much data to work with to locate them. Plus they represent a small percentage of the whole data
crimes.drop(crimes[unknown_area].index, axis = 0, inplace = True)

Now that we removed everything beyond salvation, let's focus on those records where we can find the `REPORTING_AREA` using the `STREET`, to see how many we have.

In [ ]:
unknown_area = ((crimes.LAT == -1) | (crimes.LAT.isna())) & ((crimes.LONG == -1) | (crimes.LONG.isna())) & (crimes.DISTRICT == 'UNDEFINED')
print(f"There are {crimes[unknown_area].shape[0]} records to check ({round(crimes[unknown_area].shape[0] / crimes.shape[0], 4)} %)")

In [ ]:
# let's check, for this subset, how easy it could be to search and fine tune the reporting areas based on information on the web.
print(f"There are {len(crimes.STREET[unknown_area].value_counts(dropna = False))} different street values")

Since there are so many different `STREET` names to check individually and the cost of fixing these is too high compared to the gain we can achieve (these only represent 0.0021% of the data), we will drop these from the dataset.

In [ ]:
crimes.drop(crimes[unknown_area].index, axis = 0, inplace = True)

Finally, we are missing the values that have `DISTRICT = 'UNDEFINED'` but that there's a correct `LAT` and `LONG` we could use to identify the area.

In [ ]:
crimes[crimes.DISTRICT == 'UNDEFINED'].head(10)

We will try to identify the neighbors by visually inspecting and correlating the data

In [ ]:
def generate_plot_map(df, lat_col, long_col, color_col, styles):
    """
    Creates a map figure with the data from a given dataframe 'df', which has
    a latitude column data 'lat_col' and a longitude column data 'long_col'.

    Returns the generated figure.

    It also allows to set a color column ('color_col') and some 'styles' 
    for optional plotting options
    """
    styles = { 
        **{ 
            'hover_name': None,
            'hover_data': None,
            'opacity': 0.3,
            'zoom': 10,
            'height': 400 
        }, **styles}
        
    fig = px.scatter_mapbox(df, 
                        lat = lat_col, 
                        lon = long_col,
                        hover_name = styles['hover_name'],
                        hover_data = styles['hover_data'],
                        color = color_col,
                        opacity = styles['opacity'],
                        zoom = styles['zoom'], 
                        height = styles['height'])
    fig.update_layout(mapbox_style = "open-street-map")
    fig.update_layout(margin = {"r":0,"t":0,"l":0,"b":0})

    return fig

In [ ]:
# removing any nulls in 'LAT' and 'LONG' and the outliers of -1, as we will deal with them later
correct_lat_long = ~(((crimes.LAT == -1) | (crimes.LAT.isna())) & ((crimes.LONG == -1) | (crimes.LONG.isna())))

styles = {
    'hover_name': 'OFFENSE_CODE_GROUP',
    'hover_data': ['OCCURRED_ON_DATE', 'DISTRICT'],
    'opacity': 0.5,
    'zoom': 10.5,
    'height': 500
}

district_fig = generate_plot_map(crimes[correct_lat_long], lat_col = 'LAT', long_col = 'LONG', color_col = 'DISTRICT', styles = styles)
district_fig.show()

As we can see the distribution of the position of the crimes in Boston City is pretty clear and each dot is grouped forming the shape of each city neighborhood. The approach we want to try is the following:
- For each one of the Boston districts, find its centroid, which is the dot with the minimum distance with all the points in the cloud with the already given tagged data.
- Then, for each one of the `UNDEFINED` districts, we can check the centroid with the smallest distance to it so we can then classify it accordingly to the specific district

For that purpose, we will also need to keep the centroid positions because taking into account that we have data where `LAT == -1` and `LONG == -1`, we could use those centroids in order to fill in that data later.

In that case, the best approach would be to use the `KNeighborsClassifier` algorithm to classify the missing Districts, as we are trying to solve a classification (supervised) problem with tagged data. Nonetheless, this algorithm doesn't provide the capability to calculate/return the centroids using `scikit-learn`, so we will calculate them once the classification has been done.

In [ ]:
def train_test_val_split(X, y, train_pct, test_pct, seed = 42):
    """
    Given a set of features 'X' and a set of outcomes 'y', it generates the 3 
    datasets needed to train, test and validate a model.

    The function also accepts a 'train_pct' and a 'test_pct' to be able to define the 
    percentages of rows included in each one of the train, test and validation datasets.
    For the validation dataset it uses the remaining percentage
    """

    # splitting into train + test + validation
    # in the first split, we will split the data into training and the remaining data set. We are getting 80% of all samples
    X_train, X_rem, y_train, y_rem = model_selection.train_test_split(
                                          X, 
                                          y, 
                                          train_size = train_pct,
                                          random_state = seed
                                  )     

    # since we now want to have the test and validation sizes to be equal, we will divide the sizes of X_rem by 50%
    X_val, X_test, y_val, y_test = model_selection.train_test_split(
                                          X_rem,
                                          y_rem,
                                          test_size = test_pct / (1 - train_pct),
                                          random_state = seed
                                  )
    return X_train, X_test, X_val, y_train, y_test, y_val

In [ ]:
# splitting between train, test, validation and what we want to predict
crimes_filtered_lat_long = crimes.loc[correct_lat_long] # removing all nulls and -1 in 'LAT' and 'LONG

# extracting the set we will need to predict - all 'DISTRICT's tagged as 'UNDEFINED'
X_pred = crimes_filtered_lat_long.loc[crimes.DISTRICT == 'UNDEFINED', ['LAT', 'LONG']]

# extracting the features to train, test and validate the model
X = crimes_filtered_lat_long.drop(X_pred.index, axis = 0)[['LAT', 'LONG']]
y = crimes_filtered_lat_long.drop(X_pred.index, axis = 0)[['DISTRICT']]

X_train, X_test, X_val, y_train, y_test, y_val = train_test_val_split(X, y, 0.8, 0.1, SEED)

In [ ]:
print(f"X_train size is {X_train.shape[0]} ({round(X_train.shape[0] / X.shape[0] * 100 , 2)}% of all entries)")
print(f"X_val size is {X_val.shape[0]} ({round(X_val.shape[0] / X.shape[0] * 100, 2)}% of all entries)")
print(f"X_test size is {X_test.shape[0]} ({round(X_test.shape[0] / X.shape[0] * 100, 2)}% of all entries)")

In [ ]:
# checklist
assert X.shape[0] + X_pred.shape[0] == crimes_filtered_lat_long.shape[0] # check that the size of the train + test + validation along with the to predict dataset has been correctly split
assert round(X_train.shape[0] / X.shape[0] * 100, 2) == 80 # check that training dataset represents 80% over total dataset
assert round(X_val.shape[0] / X.shape[0] * 100, 2) == 10 # check that validation dataset represents 10% over total dataset
assert round(X_test.shape[0] / X.shape[0] * 100, 2) == 10 # check that test dataset represents 10% over total dataset

In [ ]:
# Optimizing parameters - selecting the best n_neighbors
neighbors = np.arange(1, 15)
train_accuracy = np.empty(len(neighbors))
test_accuracy = np.empty(len(neighbors))
  
# Loop over K values
for i, k in enumerate(neighbors):
    print('Checking n_neighbors =', k)
    knn = KNeighborsClassifier(n_neighbors = k)
    knn.fit(X_train, y_train)
      
    # Compute traning and test data accuracy
    train_accuracy[i] = knn.score(X_train, y_train)
    test_accuracy[i] = knn.score(X_val, y_val)
  
# Generate plot
plt.plot(neighbors, test_accuracy, label = 'Validation dataset Accuracy')
plt.plot(neighbors, train_accuracy, label = 'Training dataset Accuracy')
  
plt.legend()
plt.xlabel('n_neighbors')
plt.ylabel('Accuracy')
plt.show()

In [ ]:
# using n_neighbors = 5 as it has the highest peaks for both training and validation datasets
knn = KNeighborsClassifier(n_neighbors = 5)
knn.fit(
    X_train,
    y_train
)

In [ ]:
# checking algorithm metrics
print("Training Dataset Score:  ", knn.score(X_train, y_train))
print("Validation Dataset Score:", knn.score(X_val, y_val))
print("Test Dataset Score:      ", knn.score(X_test, y_test))

The scores for each one of the datasets are pretty high and similar, with no higher differences between train, validation and test which could mean there's overfitting or underfitting, which means that the model is pretty good to be used.

In [ ]:
# predict the values from the X_pred DataFrame we retrieved
y_pred = knn.predict(X_pred)

In [ ]:
# replacing the values in 'DISTRICT' with the specific indices using the predicted y_values
crimes.loc[X_pred.index, 'DISTRICT'] = y_pred

After predicting the values of the neighborhoods, let's check that the null values and the `UNDEFINED` have been finally removed.

In [ ]:
crimes.DISTRICT.value_counts(dropna = False)

In [ ]:
crimes.DISTRICT.isna().any()

In [ ]:
styles = {
    'hover_name': 'OFFENSE_CODE_GROUP',
    'hover_data': ['OCCURRED_ON_DATE', 'DISTRICT'],
    'opacity': 0.5,
    'zoom': 10.5,
    'height': 500
}

district_fig = generate_plot_map(crimes[correct_lat_long], lat_col = 'LAT', long_col = 'LONG', color_col = 'DISTRICT', styles = styles)
district_fig.show()

We have removed the null values and also the `UNDEFINED` district, which we converted into the right neighbours. Even though there are still some data points that do not correspond to the neighbours where they have been tagged, the vast majority does, so we will leave this data treatment as finished.

`SHOOTING`

In [ ]:
crimes.SHOOTING.value_counts(dropna = False, normalize = True)

Almost all values are `NaN` (aprox 100%) and the others are `Y`. This would mean that in the database they may have like a checkbox where they only mark it if there's a shooting or not. So we can assume that all those nulls are No Shootings or `N`. 

We also need to consider that this variable has a pretty low variance, but it's important information we don't want to loose. If there are shootings, this is important for BPD to be prepared. This may not be a good variable to use in a ML model, but it's worth keeping it for future reference.

In [ ]:
crimes.SHOOTING.fillna('N', inplace = True)

`UCR_PART`

In [ ]:
crimes.UCR_PART.value_counts(dropna = False)

The best we can do here is include the `NaN` inside of `Other`.

In [ ]:
crimes.UCR_PART.fillna('Other', inplace = True)

`STREET`

In [ ]:
crimes.STREET.value_counts(dropna = False)

In [ ]:
crimes.STREET.value_counts(dropna = False, normalize = True)

Since we don't have any street reference that we can use, we will fill `NaN`s with `UNDEFINED`.

In [ ]:
crimes.STREET.fillna('UNDEFINED', inplace = True)

`LAT` and `LONG`

In [ ]:
crimes.LAT.value_counts(dropna = False)

In [ ]:
crimes.LONG.value_counts(dropna = False)

As we can see, the null values are its vast majority, but using the `DISTRICT`s, we managed to fill in a lot of records so we don't have undefined values. Using that, we can calculate the centroids of each `DISTRICT` and fill in the nulls (plus, we can also use those centroids to remove the outliers for something more accurante than the mean or the median, also known as, the center of Boston).

In [ ]:
def calculate_centroid(X, Y):
    """
    Given an array of 'x' and 'y' arrays of coordinates, returns the center the figure defined by them
    """
    return np.sum(X) / len(X), np.sum(Y) / len(Y) 

In [ ]:
# calculate centroids
districts = crimes.DISTRICT.unique()

c_latitudes = []
c_longitudes = []
for district in districts:
    X = crimes.loc[(crimes.LAT !=-1) & (~crimes.LAT.isna()) & (crimes.DISTRICT == district), 'LAT']
    Y = crimes.loc[(crimes.LONG !=-1) & (~crimes.LONG.isna()) & (crimes.DISTRICT == district), 'LONG']
    lat, long = calculate_centroid(X, Y)
    
    c_latitudes.append(lat)
    c_longitudes.append(long)

centroids = {
    'DISTRICT': districts,
    'LAT': c_latitudes,
    'LONG': c_longitudes
}
centroids_df = pd.DataFrame(centroids)

In [ ]:
styles = {
    'hover_name': 'DISTRICT',
    'opacity': 1,
    'zoom': 10.5,
    'height': 500
}

centroids_fig = generate_plot_map(centroids_df, lat_col = 'LAT', long_col = 'LONG', color_col = None, styles = styles)
centroids_fig.update_traces(marker = dict(size = 30, color = 'black'))
centroids_fig.show()

In [ ]:
# plot centroids in graph
district_fig.add_traces(centroids_fig.data[0])
district_fig.show()

In [ ]:
# setting DISTRICT as index as it will help to loop through it
centroids_df.set_index('DISTRICT', inplace = True, drop = True)

# filling NaN and correcting outliers (-1) in 'LAT' and 'LONG'
for col in ['LAT', 'LONG']:
    for district in districts:
        crimes.loc[(crimes[col].isna() | (crimes[col] == -1)) & (crimes.DISTRICT == district), col] = centroids_df.loc[district][col]

In [ ]:
print("Null values in LAT: ", crimes.LAT.isna().any())
print("Null values in LONG:", crimes.LONG.isna().any())

In [ ]:
crimes.LAT.plot(kind = 'box')

In [ ]:
crimes.LONG.plot(kind = 'box')

With the corrections we made, we solved the null issue for both columns and also, we removed the outliers for `LAT`. However, this uncovered some other new outliers for `LONG` that were too small to see when we checked before. 

In [ ]:
# checklist
assert crimes.isna().any().sum() == 0

#### 3.1.2. Handling outliers<a class="anchor" id="handling_outliers"></a>

Let's review all numerical values again to check which ones still have outliers

In [ ]:
styles = {
  'title' : 'Histograms for categorical values distributions',
  'size' : (15, 7)
}

generate_subplots(crimes.select_dtypes(include = 'number'), 'Box', 5, styles)

There are only outliers in `LONG`. Maybe these outliers are some of the dots we saw mixing with some of the different districts before. Let's check them out!

In [ ]:
def outlier_limits(df, col):
  """
  For a given 'df' 'col' it provides the lower and upper limits 
  of the 1.5 interquartile range
  """
  iqr = df[col].quantile(.75) - df[col].quantile(.25)
  low = df[col].quantile(.25) - 1.5 * iqr
  high = df[col].quantile(.75) + 1.5 * iqr
  return (low, high)

def has_outliers(df, col):
  """
  Returns True if df[col] has outliers, understood as values outside the 1.5 * IQR
  """
  limits = outlier_limits(df, col)
  return len(df.loc[(df[col] < limits[0]) | (df[col] > limits[1]), col]) != 0

In [ ]:
# check which are the outlier limits
limits = outlier_limits(crimes, 'LONG')

# checking some of these values
crimes.loc[(crimes.LONG < limits[0]) | (crimes.LONG > limits[1]), ['DISTRICT', 'LONG']]

There around 14186 entries that correspond to outliers. Luckily, thanks to what was done before with KNN all entries have a `DISTRICT` so we can apply the same correction that we did with nulls. And later, we will also check how the map looks like, just in case we improved it!

In [ ]:
# filling NaN and correcting outliers (-1) in 'LAT' and 'LONG'
while has_outliers(crimes, 'LONG'):
    for district in districts:
        crimes.loc[(crimes.LONG < limits[0]) | (crimes.LONG > limits[1]) & (crimes.DISTRICT == district), 'LONG'] = centroids_df.loc[district]['LONG']

In [ ]:
crimes.LONG.plot(kind = 'box')

Now there are no outliers, let's check the district map again, to see how our plot has improved.

In [ ]:
styles = {
    'hover_name': 'DISTRICT',
    'opacity': 1,
    'zoom': 10.5,
    'height': 500
}

districts_fig = generate_plot_map(crimes, lat_col = 'LAT', long_col = 'LONG', color_col = 'DISTRICT', styles = styles)
districts_fig.show()

Definitely, there are some issues we can see like that:
- There are some dots in the same `LONG` but in different `LAT`, which is corresponding to a correct one according to the centroid of that `DISTRICT`, but they make the map we created to look weird, even though the outliers have been removed.
- We still have some dots classified as one district but that seem to correspond to another one taking into account its coordinates.

An improvement to be considered for the future, would be to treat each `DISTRICT` subset of data as independent and analyse the outliers coming from there, so we can apply the centroids to those dots that are quite far away from its neighborhood.

#### 3.1.3. Dealing with variable types<a class="anchor" id="dealing_variable_types"></a>

In [ ]:
crimes.info()

From the list of variables, we see that there are no variables that require to be treated as:
- Numbers (`int64` and `float64`) have the right format
- `OCCURRED_ON_DATE` has already been converted to a datetime object
- Objects (Strings) are already OK as they are alphanumeric values

#### 3.1.4. Elimination of features with low variance or highly correlated with others<a class="anchor" id="#low_variance"></a>

The only field that doesn't help us in our analysis and has a low variance is the `INCIDENT_NUMBER`, so we'll remove it.

In [ ]:
# removing 'INCIDENT_ID'
crimes.drop(columns = 'INCIDENT_NUMBER', axis = 1, inplace = True)
crimes.reset_index(drop = True, inplace = True)

### MLC 3.2 - MLC 3.4: Data Transformation, Feature Engineering and Data Selection<a class="anchor" id="data_transformation_engineering_selection"></a>

At this point we have to think about what kind of unsupervised problem we want to solve in order to proceed with the data we have. Given what we have seen until now, the highest value the Boston police department can extract from this data is to try to predict the demand forecast of agents in the streets. In order to do that, we'll be focusing on `LAT` and `LONG` and the number of crimes occurring in those areas. To see areas where there is a high density of crimes reported per year, or, said in another way, where is more likely that a crime will occur.

Along with that info, BPD, will also know where they should have its offices or posts located in order to centralise crime management better, which will be the centroids of our analysis.

For the time being, we will save a copy of the current `crimes` dataframe, and we'll be moving on to extract and transform the columns for that purpose.

In [ ]:
def save_dfs_to_csv(dfs, destination_directory, prefix = ''):
    """
    Gets an array of DataFrames 'dfs' and stores them in the given 'destination_directory'.
    Files can have a 'prefix' added if set.
    Returns a list of directories where the stored files can be found
    """
    dir_list = []

    # transform dfs into a list if it's not
    if type(dfs) != 'list':
        dfs = [dfs]

    for df in dfs:
        # check if destination_folder exists and creates it otherwise       
        if not os.path.exists(destination_directory):
            print("Creating", destination_directory, "directory")
            os.makedirs(destination_directory)
        # creating the new files
        df_name = [x for x in globals() if globals()[x] is df][0] # obtain a DatFrame name (df variable name) so the user doesn't need to input it
        file_path = './' + destination_directory + '/' + df_name
        if prefix:
            file_path += '_' + prefix
        file_path += '.csv'
        if not os.path.isfile(file_path):
            print(file_path, "doesn't exist. Creating new file")         
        else:
            print("File already exists in", destination_directory)
            print("Overwriting file")
        df.to_csv(file_path)

        dir_list.append(file_path)
    return dir_list

In [ ]:
directory = 'data/processed'

save_dfs_to_csv(crimes, directory, prefix = 'processed')

In [ ]:
# selecting the potion of the crimes dataframe that the clustering algorithms will use
selected_data = crimes[['LAT', 'LONG']].copy()

# scaling data for clustering algorithms using Standard Scaler
scaler = StandardScaler()
scaled_selected_data = scaler.fit_transform(selected_data) # this is the data we'll be using from now on on the clustering algorithm

# merging the scaled data with crimes
# the original columns are not removed as they will have its usage later in the plots
location_df = pd.DataFrame(scaled_selected_data, columns = ['LAT', 'LONG'])
crimes['SCALED_LAT'] = location_df['LAT']
crimes['SCALED_LONG'] = location_df['LONG']

By looking at this plot is really complicated to see or identify the clusters the algorithms will find, taking into account that there are no clear ones to identify at plain sight.

With this, we have finished transforming the meaningful data we want to use. In terms of data transformation, we won't be transforming or using any more data for our goal. In terms of feature engineering this 2-dimensional space formed by `LAT` and `LONG` is good enough for our purposes.

## MLC 4: Modelling and Evaluation<a class="anchor" id="modelling_evaluation"></a>

In this section we will be evaluating several clustering methods in order to see which ones provide us with meaningful results. Our goal is to be able to identify those areas where crime is more concentrated and define its centers so the police knows the best way to solve crime in Boston.

### 4.1. K-Means<a class="anchor" id="kmeans"></a>

K - Means is the most basic algorithm to use for clustering, we will try its capabilities with the current dataset and see how it works. The idea behind it is that it tries to minimize the variance on each cluster.

Nonetheless, let's try first to see what the optimal number of clusters will be by using the elbow method.

In [ ]:
def kmeans_elbow_plot(X, k_range):
    """
    Returns the elbow plot of a K-Means model that will be trained by 
    the features in 'X'.

    'k_range' identifies the range number of clusters to use to train the algorithm
    """
    distortions = []
    K = range(k_range[0], k_range[1])

    for cluster_size in K:
        kmeans = KMeans(n_clusters = cluster_size, init = 'k-means++')
        kmeans = kmeans.fit(X)
        distortions.append(kmeans.inertia_)
    
    df = pd.DataFrame({'Clusters': K, 'Distortions': distortions})
    fig = (px.line(df, x = 'Clusters', y = 'Distortions', template = 'seaborn')).update_traces(mode = 'lines+markers')
    fig.show()

In [ ]:
kmeans_elbow_plot(scaled_selected_data, (1, 13)) # there are 12 districts in Boston, so let's see how it behaves if we clusterize from 1 up to the total number of the districts

By looking at the Elbow plot, the optimal number of clusters is around 5 - 6. In our case, we will be using `n_clusters = 5`.

In [ ]:
def get_each_cluster(df, model):
    """
    From a given dataframe 'df' and a clustering 'model'
    it trains and returns a numpy array containing the clusters where
    each row in 'df' is assigned to.
    """
    # transforming df features to numpy arrays
    X = df.to_numpy()
    # fit the model
    model.fit(X)
    # assign a cluster to each example
    if hasattr(model, 'predict'):
        clusters = model.predict(X)
    else:
        clusters = model.fit_predict(X)
    print(f'{len(np.unique(clusters))} different clusters have been generated')
    return clusters

def clustering_train_plot(df, features, scaled_features, model, styles):
    """
    Trains and plots a clustering 'model' using a given dataframe 'df'
    with the target 'features'. This dataframe should also have the previous
    features scaled in 'scaled_features' columns.

    Also you can send a 'styles' dictionary for plotting options
    """
    clusters = get_each_cluster(df[scaled_features], model)
    df['CLUSTERS'] = clusters
    fig = generate_plot_map(crimes, lat_col = features[0], long_col = features[1], color_col = 'CLUSTERS', styles = styles)
    fig.show()

In [ ]:
clusters = 5
kmeans = KMeans(n_clusters = clusters, init = 'k-means++')

styles = {
    'hover_name': 'DISTRICT',
    'opacity': 0.8,
    'zoom': 10.5,
    'height': 500
}

clustering_train_plot(crimes, ['LAT', 'LONG'], ['SCALED_LAT', 'SCALED_LONG'], kmeans, styles)

From the results of this clustering model, we see that the city can be optimally divided into 5 regions according to the crime groupings. Some districts share some of the clusters. With that, what we are interpreting is that these are the major areas where crime occurences can be divided by. However, that is not very interesting because we already have the district distributions which are more granular than this.

### 4.2. DBScan<a class="anchor" id="dbscan"></a>

DBSCAN Clustering (where DBSCAN is short for Density-Based Spatial Clustering of Applications with Noise) involves finding high-density areas in the domain and expanding those areas of the feature space around them as clusters.

As from its description it seems that this may be a more promising algorithm to find the areas where the crime density is higher because it will generate clusters where the crime density is higher, or saying it in another way, areas where there are more occurrences.

In [ ]:
epsilon = 2e-4
min_samples = 5

dbscan = DBSCAN(eps = epsilon, min_samples = min_samples)

styles = {
    'hover_name': 'DISTRICT',
    'opacity': 0.8,
    'zoom': 10.5,
    'height': 500,

}

clustering_train_plot(crimes, ['LAT', 'LONG'], ['SCALED_LAT', 'SCALED_LONG'], dbscan, styles)

By using these values of epsilon and min_samples we see there are a ton of blue clusters we found there are more than 10453 clusters. The groupings of these clusters can be interpreted as the rows at a distance epsilon that at least have 5 close neighbours. These clusters identify regions of the map that have had, during all these years, a concentration of crimes higher than 5, and which represent areas where the police have in the radar.

Some clusters have the value -1, which means that these can't be grouped into close neighbours, so they are isolated cases.

Even though the plot looks messy, it's giving us a lot more information than the one K-Means gave us, because now we know which are the hot areas in Boston.

Let's take up a look at the clusters that aggregate more offenses to see where we should take a closer look at.

In [ ]:
crimes_grouped_by_clusters = crimes.groupby('CLUSTERS').count().sort_values('OFFENSE_CODE', ascending = False)
crimes_grouped_by_clusters

There are 15663 offenses whose locations are isolated and can't be grouped, let's analyse the others

In [ ]:
# removing the -1 cluster, which are isolated results
crimes_grouped_by_clusters = crimes_grouped_by_clusters[1:]

# keeping the top 100 clusters
top_50_clusters = crimes_grouped_by_clusters[:50].index

In [ ]:
styles = {
    'hover_name': 'OFFENSE_CODE_GROUP',
    'hover_data': ['OCCURRED_ON_DATE', 'DISTRICT'],
    'opacity': 1,
    'zoom': 10.5,
    'height': 500
}

fig = generate_plot_map(crimes[crimes.CLUSTERS.isin(top_50_clusters)], lat_col = 'LAT', long_col = 'LONG', color_col = 'CLUSTERS', styles = styles)
fig.update_traces(marker = dict(size = 30))
fig.show()

By doing that we can see the areas where more crimes have occurred and where the police should be more focused at.

### 4.3. Birch<a class="anchor" id="birch"></a>

BIRCH Clustering (BIRCH is short for Balanced Iterative Reducing and Clustering using Hierarchies) involves constructing a tree structure from which cluster centroids are extracted.

BIRCH incrementally and dynamically clusters incoming multi-dimensional metric data points to try to produce the best quality clustering with the available resources (i.e., available memory and time constraints). BIRCH can typically find a good clustering with a single scan of the data, and improve the quality further with a few additional scans. BIRCH is also the first clustering algorithm proposed in the database area to handle "noise" (data points that are not part of the underlying pattern) effectively.

For our problem, Birch could prove worthy in terms of efficiency in dealing with "noise" and also with the usage of resources.

In [ ]:
threshold = 0.01
clusters = 5 # we'll try with the same ones as with K-Means

birch = Birch(n_clusters = clusters, threshold = threshold)

styles = {
    'hover_name': 'DISTRICT',
    'opacity': 0.8,
    'zoom': 10.5,
    'height': 500
}

clustering_train_plot(crimes, ['LAT', 'LONG'], ['SCALED_LAT', 'SCALED_LONG'], birch, styles)

We found similar results to what we got with K-Means, so there was not a big improvement.

The most interesting clustering algorithm for our use case was, without any doubt, the DBSCAN algorithm.

Notes after trying other models:
- OPTICS (short for Ordering Points To Identify the Clustering Structure), which would have been a great option to use as it is an evolution of DBSCAN, didn't converge to a result even after fine tuning the parameters.
- Meanshift clustering was running for more than 30 minutes before I had to stop it. It's not included as it's not converging into a model that we can use for this data.

## E. Next steps - What to try next<a class="anchor" id="next_steps"></a>

- Find a better way to deal with the outliers in `LONG` to avoid the weird effect these cause in the scatter plot. Some ideas are:
  - Instead of using the centroids, use the outlier limits as values. By doing this the introduced error will be lower as we won't be assuming these occur in the same `LONG` coordinate of the centroid.
  - Split each neighbor into its own dataset, and treat the outliers locally. By doing this, it is possible that we can even center the dispersed dots closer to the neighbor itself.